-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Stored Views

In this notebook, we'll give a quick overview of how stored views are created and managed.

The goal of this notebook is to generate a view that allows the analysts from our partner gyms to examine how use of Moovio devices and workouts impact gym activity.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_gym_report.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Display the query plan associated with a view
- Describe how results are materialized from Delta Lake tables

In [0]:
%run ../Includes/gold-setup

A helper function has been defined to process a new batch of data to the source tables used in this lesson. (Note: this may take around 2 minutes.)

In [0]:
process_gold_sources()

## Examine Gym Logs

Start by reviewing the schema for your gym logs.

In [0]:
gymDF = spark.table("gym_mac_logs")
gymDF.printSchema()

A Spark DataFrame and a view are nearly identical constructs. By calling `explain` on our DataFrame, we can see that our source table is a set of instructions to deserialize the files containing our data.

In [0]:
gymDF.explain("formatted")

## Examine Workout Data

Rather than trying to capture every possible metric in our view, we'll create a summary of values that might be of interest to our gym analysts.

The data we're receiving from our gyms indicates the first and last timestamp recorded for user devices, indicated by mac address.

In [0]:
%sql
SELECT * FROM gym_mac_logs

Our `completed_workouts` table indicates start and stop time for each user workout.

Use the cell below to construct a query that identifies:
- Each date a user completed at least one workout
- The earliest `start_time` for any workout each day
- The latest `end_time` for any workout each day
- The list of all workouts completed by a user each day

In [0]:
%sql
MAGIC -- TODO
MAGIC SELECT * 
MAGIC FROM completed_workouts
MAGIC LIMIT 10

Now we'll join this data back to the MAC logs sent by the gym to create our view.

We'll retain the `mac_address` as our identifier, which we can grab from the `user_lookup` table.

We'll also add columns to calculate the total number of minutes elapsed during a user's visit to the gym, as well as the total number of minutes elapsed between the beginning of their first workout and the end of their final workout.

In [0]:
%sql
CREATE VIEW IF NOT EXISTS gym_user_stats AS (
SELECT gym, mac_address, date, workouts, (last_timestamp - first_timestamp)/60 minutes_in_gym, (to_unix_timestamp(end_workout) - to_unix_timestamp(start_workout))/60 minutes_exercising
FROM gym_mac_logs c
INNER JOIN (
  SELECT b.mac_address, to_date(start_time) date, collect_set(workout_id) workouts, min(start_time) start_workout, max(end_time) end_workout
      FROM completed_workouts a
      INNER JOIN user_lookup b
      ON a.user_id = b.user_id
      GROUP BY mac_address, to_date(start_time)
  ) d
  ON c.mac = d.mac_address AND to_date(CAST(c.first_timestamp AS timestamp)) = d.date)

We can see that our view is simply storing the Spark plan for our query.

In [0]:
spark.table("gym_user_stats").explain("formatted")

When we execute a query against this view, we will process the plan to generate the logically correct result.

Note that while the data may end up in the Delta Cache, this result is not guaranteed to be persisted, and is only cached for the currently active cluster.

In [0]:
%sql
SELECT *
FROM gym_user_stats
WHERE gym = 5

## An Aside on ACLs

While Databricks has extensive support for ACLs, by default these are not enforced for standard data engineering clusters. As such, the default permissions for this view are set to all users, and no owner has been declared.

In [0]:
import py4j

try:
  spark.sql("SHOW GRANT ON VIEW gym_user_stats")

except py4j.protocol.Py4JJavaError as e:
  print("Error: " + e.java_exception.getMessage())
  print("Solution: Consider enabling Table Access Control to demonstrate this feature.")


While the privileges for this view may not be especially sensitive, we can see that our bronze table (which contains ALL our raw data) is also currently stored in this fashion.

Again, ACLs are primarily intended for managing data access within the Databricks workspace for BI and data science use cases. For sensitive data engineering data, you will want to make sure that you limit access to the storage containers using your cloud identity management.

In [0]:
try:
  spark.sql("SHOW GRANT ON TABLE bronze")
  
except py4j.protocol.Py4JJavaError as e:
  print("Error: " + e.java_exception.getMessage())
  print("Solution: Consider enabling Table Access Control to demonstrate this feature.")

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>